# Problem 2

When you need to modify an entire table, for example, when you don't need a specific dependency (foreign key).


In [3]:
import datajoint as dj

dj.Schema("my_custom_workflow_child").drop(force=True)
dj.Schema("my_custom_workflow_child_ver2").drop(force=True)
dj.Schema("my_custom_workflow_parent").drop(force=True)


See corresponding code from file `./src/my_dj_package/child_ver2.py` or at the end of this notebook.

Notice how the databse name uses the module name.


In [5]:
from my_dj_package.child_ver2 import Child, Parent
dj.list_schemas()


['my_custom_workflow_child_ver2',
 'my_custom_workflow_parent',
 'mysql',
 'performance_schema',
 'sys']

Add manual entries, then populate downstream content.

**Note**. Not inserting anything into the `Sibling` table, as it is not used.

In [6]:
Parent.insert1({"field": "parent"})
Child.populate()


The table no longer has field2 from the `Sibling` dependency

In [7]:
Child()


field,path
parent,/Users/joseph


### Content of `./src/my_dj_package/child_ver2.py`

In [ ]:
from pathlib import Path

import datajoint as dj
from datajoint_extension.pipeline.parent import Parent
from datajoint_extension.pipeline.parent import schema as parent_schema
from datajoint_extension.utils.schema import LazySchema

# new schema will have the same prefix defined in dj.config
schema = LazySchema()


# don't want to depend on sibling
@schema
class Child(dj.Computed):
    definition = """
    -> Parent
    path : VARCHAR(512)
    """

    def make(self, key):
        key |= {"path": Path.home()}
        self.insert1(key)


parent_schema.activate()
schema.activate()
